In [1]:
import numpy as np
import pandas as pd
import re
import glob

In [12]:
glob.glob('./Second pass model outputs/*.txt')

['./Second pass model outputs\\atlantic_crossword_test.txt',
 './Second pass model outputs\\ByT5 Console Output.txt',
 './Second pass model outputs\\byt5 inference clean data.txt',
 './Second pass model outputs\\byt5 inference data.txt',
 './Second pass model outputs\\byt5 multi iteration data.txt',
 './Second pass model outputs\\newsday_crossword_test.txt',
 './Second pass model outputs\\nyt_crossword_test.txt',
 './Second pass model outputs\\T5 Console Output.txt',
 './Second pass model outputs\\t5 full dateaset word segment.txt',
 './Second pass model outputs\\t5 inference data.txt',
 './Second pass model outputs\\t5 multi iteration 70 percentage data trained on.txt',
 './Second pass model outputs\\t5 multi iteration data segmented answer model.txt',
 './Second pass model outputs\\t5 multi iteration data segmented answer model_2.txt',
 './Second pass model outputs\\t5 multi iteration data word segmented.txt',
 './Second pass model outputs\\t5 multi iteration data.txt',
 './Second pa

In [43]:
def extract_float(input_string):
    pattern = r"\d+\.\d+"
    matches = re.findall(pattern, input_string)
    float_numbers = [float(match) for match in matches]
    return float_numbers

def extract_data(lines):
    date_pattern = r"\b\d{2}/\d{2}/\d{4}\b"
    data_dict = {}
    new_date_math = False
    current_date = ''
    error_dates = []

    for line in lines:
        if 'error' in line:
            match = re.search(date_pattern, line)
            error_dates.append(match.group())
    
    for line in lines:
        match = re.match(date_pattern, line) # to match the date pattern
        if match:
            current_date = line.strip()
            data_dict[current_date] = {}
            data_dict[current_date]['Letter II'] = []
            data_dict[current_date]['Word II'] = []

        if 'Before' in line:
            [lett_accu, word_accu] = extract_float(line)
            data_dict[current_date]['Before Letter Accuracy'] = lett_accu
            data_dict[current_date]['Before Word Accuracy'] = word_accu

        if 'iteration:' in line:
            lett_accu, word_accu = extract_float(line)
            data_dict[current_date]['Letter II'].append(lett_accu)
            data_dict[current_date]['Word II'].append(word_accu)
            
    return data_dict

In [44]:
def get_df_data(data_dict, threshold = 85.0):
    output_data = []
    for date, inf_data in data_dict.items():

        if len(inf_data.keys()) < 4:
            continue

        # only first pass model output exists
        if len(inf_data['Letter II']) == 0:
            f_pass_l_accu = round(inf_data['Before Letter Accuracy'], 2)
            f_pass_w_accu = round(inf_data['Before Word Accuracy'], 2)

            s_pass_l_accu = f_pass_l_accu
            s_pass_w_accu = f_pass_w_accu
            output_data.append((date, f_pass_l_accu, f_pass_w_accu, s_pass_l_accu, s_pass_w_accu))
        else:
            f_pass_l_accu = round(inf_data['Before Letter Accuracy'], 2)
            f_pass_w_accu = round(inf_data['Before Word Accuracy'], 2)

            s_pass_l_list = inf_data['Letter II']
            s_pass_w_list = inf_data['Word II']
            max_accu_index = s_pass_l_list.index(max(s_pass_l_list))

            s_pass_l_accu = round(s_pass_l_list[max_accu_index], 2)
            s_pass_w_accu = round(s_pass_w_list[max_accu_index], 2)

            if s_pass_l_accu < f_pass_l_accu:
                s_pass_l_accu = f_pass_l_accu
                s_pass_w_accu = f_pass_w_accu

            output_data.append((date, f_pass_l_accu, f_pass_w_accu, s_pass_l_accu, s_pass_w_accu))
            
    df = pd.DataFrame(output_data, columns = ['Date', 'First Pass Letter Accuracy', 'First Pass Word Accuracy', 'Second Pass Letter Accuracy', 'Second Pass Word Accuracy'])
    return df

In [45]:
hard_path = "./Second pass model outputs/t5_small new dataset trained - tested on hard solutions.txt"
lines = open(hard_path, 'r').readlines()
data_dict = extract_data(lines)
df_ = get_df_data(data_dict)
# df_.to_csv("./Second pass model outputs/hard solution with new t5_small.csv")
df_.describe()

,First Pass Letter Accuracy,First Pass Word Accuracy,Second Pass Letter Accuracy,Second Pass Word Accuracy
count,24.000000,24.000000,24.000000,24.000000
mean,98.515833,94.445000,99.292083,97.507917
std,1.792094,5.911622,1.612422,5.201537
min,93.440000,79.490000,93.440000,79.490000
25%,97.847500,91.142500,99.485000,97.107500
50%,99.210000,96.300000,100.000000,100.000000
75%,100.000000,100.000000,100.000000,100.000000
max,100.000000,100.000000,100.000000,100.000000


In [46]:
path = "./Second pass model outputs/t5 multi iteration wod segmented 2 epochs.txt"
lines = open(path, 'r').readlines()
data_dict = extract_data(lines)
df_ = get_df_data(data_dict)
df_.describe()

,First Pass Letter Accuracy,First Pass Word Accuracy,Second Pass Letter Accuracy,Second Pass Word Accuracy
count,24.000000,24.000000,24.000000,24.000000
mean,97.294167,90.094583,99.156250,96.721250
std,1.854412,5.981114,1.303533,4.602359
min,92.390000,77.140000,95.630000,84.620000
25%,96.150000,85.462500,98.912500,95.410000
50%,97.840000,90.675000,99.745000,98.610000
75%,98.367500,92.670000,100.000000,100.000000
max,100.000000,100.000000,100.000000,100.000000
